![wompi](Wompi.png)
# **Prueba Técnica - Especialista analítica de negocio**
## Prueba Analítica: *Análisis de segmentación de comercios*
### Cientifico de Datos:

* Daniel Felipe Pérez Grajales . dfperezg@unal.edu.co<br>

<br><br>


---
<br>

**Objetivo:**

Realizar un análisis de segmentación de comercios a partir de datos transaccionales, identificando grupos con comportamientos similares y detectando oportunidades comerciales dentros de la industria.

**Instrucciones**

Se te entrega un archivo *payments_datasets.csv* con información de transacciones de pago Deberás:

1. Construir métricas a nivel de comercio como: número de transacciones, ticket promedio, tasa de aprobación y distribución de medios de pago.

2. Preparar el conjunto de datos para clustering, estandarizando variables y seleccionando las más relevantes para la segmentación.

3. Aplicar una técnica de clustering.

4. Analizar y describir los clusters, resaltando similitudes y diferencias entre ellos.

5. Detectar oportunidades comerciales dentro de cada industrial(merchant_ciiu), comparando comercios con caracteristicas similares pero diferencias marcadas en el uso de medios de pago.

**Entregables**

Debes entregar un archivo Jupyter Notebook (.ipynb) funcional que cumpla con estos pasos:


* Carga del dataset.  
* Cálculo de métricas por comercio.
* Ejecución del algoritmo de clustering y elección del número óptimo de clusters.
* Tabla de oportunidades comerciales mostrando pares de comercios comparables con brechas significativas en uso de medios de pago.
* Conclusiones y recomendaciones orientadas a negocio.



--------

<br>
Solución Análisis de segmentación:

0. Carga de Módulos.<br>
1. Carga del dataset.<br>
2. Cálculo de métricas por comercio.<br>
3. Ejecución del algoritmo de clustering y elección del número óptimo de clusters.<br>
4. Tabla de oportunidades comerciales mostrando pares de comercios comparables con brechas significativas en uso de medios de pago.<br>
5. Conclusiones y recomendaciones orientadas a negocio.

### o. Carga de Modulos requeridos

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado clusterin
# ==============================================================================
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import scale

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

### 1. Carga del dataset.

In [2]:
df_trans_comer= pd.read_csv(r'./payments_dataset.csv', sep=',')
print(df_trans_comer.shape)
df_trans_comer.head()

(12000, 8)


,id,merchant_id,merchant_name,merchant_ciiu,amount_in_cents,created_date,payment_method_type,status
0,1000000,1040,Comercio_1040,G4741,78240,2025-05-08 08:02:36,NEQUI,APPROVED
1,1000001,1007,Comercio_1007,I5610,21469,2025-04-03 20:33:29,BANCOLOMBIA,APPROVED
2,1000002,1001,Comercio_1001,G4711,73617,2025-06-27 15:29:33,PSE,APPROVED
3,1000003,1047,Comercio_1047,G4761,40190,2025-04-05 00:05:00,PSE,REFUNDED
4,1000004,1017,Comercio_1017,G4773,107172,2025-06-24 21:22:58,NEQUI,APPROVED


* ID
* ID del comerciante
* Nombre del comerciante
* CII del comerciante
* Importe en centavos
* Fecha de creación
* Tipo de método de pago
* Estado

In [3]:
df_trans_comer.describe(include='all')

,id,merchant_id,merchant_name,merchant_ciiu,amount_in_cents,created_date,payment_method_type,status
count,1.200000e+04,12000.000000,12000,12000,1.200000e+04,12000,11280,12000
unique,NaN,NaN,50,10,NaN,11998,5,3
top,NaN,NaN,Comercio_1043,G4741,NaN,2025-06-30 19:13:32,NEQUI,APPROVED
freq,NaN,NaN,271,1294,NaN,2,2504,10528
mean,1.006000e+06,1024.834000,NaN,NaN,7.808565e+04,NaN,NaN,NaN
std,3.464246e+03,14.418776,NaN,NaN,8.228226e+04,NaN,NaN,NaN
min,1.000000e+06,1000.000000,NaN,NaN,4.060000e+03,NaN,NaN,NaN
25%,1.003000e+06,1012.000000,NaN,NaN,3.143700e+04,NaN,NaN,NaN
50%,1.006000e+06,1025.000000,NaN,NaN,5.084900e+04,NaN,NaN,NaN
75%,1.008999e+06,1037.000000,NaN,NaN,9.005725e+04,NaN,NaN,NaN


### cantidad de comercios

In [8]:
df_trans_comer['merchant_name'].nunique()

50

### cantidad de transacciones por comercios

In [6]:
print(df_trans_comer['merchant_name'].value_counts(dropna=False))
print()
print(df_trans_comer['merchant_name'].value_counts(True,dropna=False)*100)

merchant_name
Comercio_1043    271
Comercio_1041    264
Comercio_1005    263
Comercio_1036    262
Comercio_1044    261
Comercio_1008    259
Comercio_1035    255
Comercio_1028    255
Comercio_1031    254
Comercio_1019    252
Comercio_1020    252
Comercio_1049    251
Comercio_1042    250
Comercio_1029    250
Comercio_1040    248
Comercio_1030    247
Comercio_1024    246
Comercio_1026    245
Comercio_1007    245
Comercio_1023    244
Comercio_1009    244
Comercio_1034    244
Comercio_1047    243
Comercio_1039    242
Comercio_1015    241
Comercio_1048    240
Comercio_1006    239
Comercio_1002    237
Comercio_1018    237
Comercio_1021    236
Comercio_1038    235
Comercio_1003    234
Comercio_1046    233
Comercio_1037    233
Comercio_1045    232
Comercio_1033    231
Comercio_1027    230
Comercio_1012    230
Comercio_1004    229
Comercio_1011    229
Comercio_1025    229
Comercio_1010    226
Comercio_1014    226
Comercio_1017    225
Comercio_1016    224
Comercio_1032    223
Comercio_1022    217

### Canales de pago

In [9]:
print(df_trans_comer['payment_method_type'].value_counts(dropna=False))
print()
print(df_trans_comer['payment_method_type'].value_counts(True,dropna=False)*100)

payment_method_type
NEQUI          2504
BANCOLOMBIA    2468
CARD           2288
CASH           2243
PSE            1777
NaN             720
Name: count, dtype: int64

payment_method_type
NEQUI          20.866667
BANCOLOMBIA    20.566667
CARD           19.066667
CASH           18.691667
PSE            14.808333
NaN             6.000000
Name: proportion, dtype: float64


### Estados de las transacciones

In [11]:
df_trans_comer.dtypes

id                      int64
merchant_id             int64
merchant_name          object
merchant_ciiu          object
amount_in_cents         int64
created_date           object
payment_method_type    object
status                 object
dtype: object

In [10]:
print(df_trans_comer['status'].value_counts(dropna=False))
print()
print(df_trans_comer['status'].value_counts(True,dropna=False)*100)

status
APPROVED    10528
DECLINED     1212
REFUNDED      260
Name: count, dtype: int64

status
APPROVED    87.733333
DECLINED    10.100000
REFUNDED     2.166667
Name: proportion, dtype: float64


### 2. Cálculo de métricas por comercio.

* número de transacciones.
* ticket promedio.
* tasa de aprobación y distribución de medios de pago.

In [13]:
df_trans_comer.head(3)

,id,merchant_id,merchant_name,merchant_ciiu,amount_in_cents,created_date,payment_method_type,status
0,1000000,1040,Comercio_1040,G4741,78240,2025-05-08 08:02:36,NEQUI,APPROVED
1,1000001,1007,Comercio_1007,I5610,21469,2025-04-03 20:33:29,BANCOLOMBIA,APPROVED
2,1000002,1001,Comercio_1001,G4711,73617,2025-06-27 15:29:33,PSE,APPROVED


In [12]:
df_trans_comer.dtypes

id                      int64
merchant_id             int64
merchant_name          object
merchant_ciiu          object
amount_in_cents         int64
created_date           object
payment_method_type    object
status                 object
dtype: object

### construcción de variables fecha

In [14]:
import datetime
df_trans_comer['created_date']=pd.to_datetime(df_trans_comer['created_date'],format='%Y-%m-%d %H:%M:%S',errors='coerce')#datetime.date.today()
df_trans_comer['anio_mes'] = df_trans_comer['created_date'].dt.strftime('%Y%m') 
df_trans_comer.head()

,id,merchant_id,merchant_name,merchant_ciiu,amount_in_cents,created_date,payment_method_type,status,anio_mes
0,1000000,1040,Comercio_1040,G4741,78240,2025-05-08 08:02:36,NEQUI,APPROVED,202505
1,1000001,1007,Comercio_1007,I5610,21469,2025-04-03 20:33:29,BANCOLOMBIA,APPROVED,202504
2,1000002,1001,Comercio_1001,G4711,73617,2025-06-27 15:29:33,PSE,APPROVED,202506
3,1000003,1047,Comercio_1047,G4761,40190,2025-04-05 00:05:00,PSE,REFUNDED,202504
4,1000004,1017,Comercio_1017,G4773,107172,2025-06-24 21:22:58,NEQUI,APPROVED,202506


### transacciones por mes

In [15]:
print(df_trans_comer['anio_mes'].value_counts(dropna=False))
print()
print(df_trans_comer['anio_mes'].value_counts(True,dropna=False)*100)

anio_mes
202503    2098
202507    2061
202505    2059
202506    2028
202504    2010
202502     897
202508     847
Name: count, dtype: int64

anio_mes
202503    17.483333
202507    17.175000
202505    17.158333
202506    16.900000
202504    16.750000
202502     7.475000
202508     7.058333
Name: proportion, dtype: float64


### Meticas generales

In [ ]:
BD_metri_come=df_trans_comer.groupby(['merchant_name','merchant_ciiu']).size().reset_index(name='cant_transs' \
'' \
'')

In [ ]:
BD_grupos=BD_algo2.groupby(['NumeroDocIdCliente','merchant_ciiu']).size().reset_index(name='frec_grupo')

BD_grupos=BD_grupos.sort_values(['NumeroDocIdCliente','frec_grupo'],ascending=[True,False]).groupby('NumeroDocIdCliente').head(1)
BD_grupos.shape

BDA= BD_algo2_fn[(BD_algo2_fn['Grupo_categ']=="A")].pivot_table(values='score', index='categoria', columns='NumeroDocIdCliente', fill_value=0)
print(BDA.shape)

BD_recom_A['Top3_categorias'] = BD_recom_A.sort_values(['NumeroDocIdCliente','categoria'],ascending=[True,True]).groupby(['NumeroDocIdCliente'])['categoria'].transform(lambda x : '|'.join(x))
BD_recom_A['Orden_categ'] = BD_recom_A.groupby(['NumeroDocIdCliente'])['categoria'].transform(lambda x : '|'.join(x))


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts

[Document(metadata={'source': '/content/bre-b.txt'}, page_content='La interoperabilidad bancaria en Colombia ha ganado relevancia en los últimos años. Pero ¿qué significa exactamente y cómo hace parte del día a día de las personas?\xa0\n¿Qué es la interoperabilidad bancaria?\nLa interoperabilidad en el sistema financiero es la capacidad de hacer transacciones financieras de manera fluida y eficiente a través de la interconexión entre diferentes entidades que hacen parte de la industria financiera.\xa0\n\xa0Imagina que tienes una cuenta en el\xa0Banco A, pero necesitas hacer un pago o transferencia a alguien que tiene una cuenta en el\xa0Banco B. Antes, este proceso podía ser complicado y lento debido a los sistemas y procesos de cada banco.\xa0\xa0\n\xa0Por medio de la interoperabilidad, puedes hacer esta transacción de manera rápida y sencilla, sin importar en qué entidad tengas tu cuenta. Este concepto no es nuevo en el país; ejemplos como\xa0Transfiya\xa0y\xa0PSE\xa0demuestran que l

### 3. Ejecución del algoritmo de clustering y elección del número óptimo de clusters.

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

/tmp/ipython-input-6-2665919207.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### vectorizar texto

In [7]:
vectordb = Chroma.from_documents(texts, embedding)
vectordb


# Realizar una búsqueda simple en el documento por similitud

In [8]:
# **Search**
query = "¿Qué es la interoperabilidad bancaria?"
results = vectordb.similarity_search(query, k=3)
for result in results:
    print(f"Document: {result.page_content}")

Document: Latinoamérica es una región que se ha caracterizado por tener un amplio uso del efectivo y porque muchas personas aún no tienen una cuenta en el banco o en billeteras digitales como Nequi, pero esta realidad ha cambiado y cada vez son más las posibilidades para que los pagos se hagan en minutos y sin complicaciones.
Dentro de este contexto, las transacciones en línea también juegan un papel importante, con pocos clics pagas el recibo de la luz, alguna compra en internet o disfrutas de tu película favorita. La facilidad con la que se hacen estos pagos y cómo lo usamos en la vida diaria ha hecho que las empresas del sector financiero hablen cada vez más de interoperabilidad.
¿Interoperabilidad? ¿Eso qué es?
Document: Muchas veces, las transferencias entre cuentas bancarias pueden demorarse días y, si haces tus transacciones los fines de semana o festivos, deberás esperar hasta el lunes o martes para que sea efectiva. Así, que aprovecha las ventajas que te dará Bre-B para mover 

### 4. Tabla de oportunidades comerciales mostrando pares de comercios comparables con brechas significativas en uso de medios de pago.

In [9]:
#Login to Hugginface
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [10]:
!huggingface-cli whoami

dfperezg
orgs:  The0rganization


In [11]:
import transformers
from transformers import AutoTokenizer
import torch
from torch import cuda, bfloat16

### Se Establezca la configuración de cuantificación para un menor uso de memoria de la GPU

In [12]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Carga de modelos Free LLM Huggis face

In [62]:
model_id = "facebook/bart-large-cnn"#"google-bert/bert-base-uncased""facebook/bart-large-cnn" posibles modelos free LLMs

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)#, use_auth_token=True

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:935: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


### 5. Conclusiones y recomendaciones orientadas a negocio.

In [77]:
#Model Pipeline

from transformers import pipeline

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    # we pass model parameters here too
    # stopping_criteria=stopping_criteria,  # without this model will ramble
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=500,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
    # do_sample=True,
    # truncation=True,
    # padding='max_length',
    # num_return_sequences=2,
    # temperature=1.0,
    # num_beams=1,
    # max_length=1024,
    # max_new_tokens=512,
)

Device set to use cpu


In [78]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

In [16]:
llm(prompt="What is Sentiment analysis")

/tmp/ipython-input-16-2625105724.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(prompt="What is Sentiment analysis")


'What is Sentiment analysis overra SheoTheseraoo youo oneraorao dorara over downase over over do over over over down over over Take over over their over over only over over one over over at down over only down over at over over a one over at only over do one over down at over do at over at do over do just only over onlyke over do down over down/ over over is over atather over at/ down over Che over overashaterash over at atater over over Che at at overash Che down Che downater downateraterash Che over ataterater overater over Cheaterashaterker Cheater Che over'

Inicializar cadena

In [79]:
from langchain.chains import RetrievalQA

### 6. Sección realizar preguntas al documento que responda con referencia a los chunks recuperado/utilizados.

In [80]:
retriever = vectordb.as_retriever()

retrieve = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [81]:
def rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)

In [82]:
prompt = "Que es la interoperabilidad bancaria"
rag(retrieve, prompt)

Both `max_new_tokens` (=500) and `max_length`(=143) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Query: Que es la interoperabilidad bancaria



> Entering new RetrievalQA chain...


IndexError: index out of range in self

--------------------------
Fin
